In [1]:
import ee
import folium

# # opens browser to request authenticatication token (OAUTH2) and store it.
# ee.Authenticate()

# uses your stored auth token to login to ee.
ee.Initialize()

class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self


In [2]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2014,
    attr="Lior"
)

emap


In [3]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2014,
    attr="Lior",
    vis_params={
        "bands": ['treecover2000'],
    },
)

emap

In [4]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2014,
    attr="Lior",
    vis_params={
        "bands": ['last_b50', 'last_b40', 'last_b30'],
    },
)

emap

In [5]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2014,
    attr="Lior",
    vis_params={
        "bands": ['loss', 'treecover2000', 'gain'],
    },
)

emap

In [6]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2014,
    attr="Lior",
    vis_params={
        "bands": ['loss', 'treecover2000', 'gain'],
        "max": [1,255,1],
    },
)

emap

In [7]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2014,
    attr="Lior",
    vis_params={
        "bands": ['treecover2000'],
        "palette": ['000000', '00FF00']
    },
)

emap

In [8]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2014,
    attr="Lior",
    vis_params={
        "bands": ['treecover2000'],
        "palette": ['000000', '00FF00'],
        "max": 100,
    },
)

emap

In [9]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2014.mask(gfc2014),
    attr="Lior",
    vis_params={
        "bands": ['treecover2000'],
        "palette": ['000000', '00FF00'],
        "max": 100,
    },
)

emap

In [10]:
# load a dataset image
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

treeCover = gfc2014.select(['treecover2000']);
lossImage = gfc2014.select(['loss']);
gainImage = gfc2014.select(['gain']);

# make a map
emap = EngineMap()

# add treecover in green
emap.add_ee_raster(
    image=treeCover.updateMask(treeCover),
    name='ForestCover',
    attr="Lior",
    vis_params={
        "palette": ['000000', '00FF00'],
        "max": 100,
    },
)

# add loss in red
emap.add_ee_raster(
    image=lossImage.updateMask(lossImage),
    name='Loss',
    attr="Lior",
    vis_params={
        "palette": ['FF0000'],
    },
)

# add gain in blue
emap.add_ee_raster(
    image=gainImage.updateMask(gainImage),
    name='Gain',
    attr="Lior",
    vis_params={
        "palette": ['0000FF'],
    },
)

emap